In [3]:
# Unimi 2023/2024
# Progetto di Visualizzazione Scientifica
# 984605 Pomayay Gabonal Angello Fernando 
# 01553A Mbaye Serigne Darou

import pandas as pd
import plotly.express as px

# Soppressione degli avvertimenti
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Lettura del CSV
df = pd.read_csv("../movies_r.csv")

# Conversione della colonna 'Year' in formato datetime
df['Year'] = pd.to_datetime(df['Year'], format='%Y')

# Rimuovo gli spazi extra dalla colonna 'Genre'
df['Genre'] = df['Genre'].str.strip()

# Ordina il DataFrame per la colonna 'Year' in modo da ottenere dati consecutivi senza picchi strani nel grafico
df = df.sort_values(by='Year')

# Espansione dei generi separati da virgola in nuove righe
df_expanded = df.assign(Genre=df['Genre'].str.split(',')).explode('Genre')

# Rimozione degli spazi extra prima e dopo ciascun genere
df_expanded['Genre'] = df_expanded['Genre'].str.strip()

# Calcolo la somma annuale dei voti per ogni genere
df_expanded = df_expanded.groupby(['Year', 'Genre'], as_index=False).agg({'Votes': 'sum'})

# Seleziono i primi 5 generi come quelli inizialmente visibili
generi_selezionati = df_expanded['Genre'].unique()[:5]

# Creazione del grafico
fig = px.line(df_expanded, x='Year', y='Votes', color='Genre',
              labels={'Votes': 'Popolarità', 'Year': 'Anno'},
              title='Andamento della popolarità per genere nel corso degli anni',
              color_discrete_sequence=px.colors.qualitative.Pastel,
              template='plotly_white') 

# Imposto la visibilità iniziale dei generi
for genere in df_expanded['Genre'].unique():
    fig.for_each_trace(lambda trace: trace.update(visible=True) if trace.name in generi_selezionati else trace.update(visible='legendonly'))
    
fig.update_layout(width=1400, height=800)
fig.write_html("../file_html/genere_popolarita.html")
fig.show()